# T-Maze Inference for Planning with (C)BFE

This notebook reports the (C)BFE for all policies for a variety of scenarios, and is used to generate Fig. 7 (Sec. 6.2).

In [43]:
using LinearAlgebra
using ForneyLab
using Plots

# T-maze layout
# [2| |3]
#   | |
#   |1|
#   |4|

# Constrained Model

In [44]:
T = 2

fg_constrained = FactorGraph()

u = Vector{Variable}(undef, T)
x = Vector{Variable}(undef, T)
y = Vector{Variable}(undef, T)

@RV x_t_min ~ Categorical(placeholder(:D_t_min, dims=(8,)))

x_k_min = x_t_min
for k=1:T
    @RV u[k]
    @RV x[k] ~ Transition(x_k_min, u[k])
    @RV y[k] ~ Transition(x[k], placeholder(:A, dims=(16,8), var_id=:A_*k))

    placeholder(u[k], :u, index=k, dims=(8,8))
    Categorical(y[k], placeholder(:C, dims=(16,), index=k, var_id=:C_*k))
    PointMassConstraint(y[k])
    
    x_k_min = x[k]
end
;

In [45]:
q_plan_constrained = PosteriorFactorization(y, [x_t_min; x], ids=[:Y, :X])
algo_plan_constrained = messagePassingAlgorithm(y, id=:PlanConstrained, free_energy=true)
code_plan_constrained = algorithmSourceCode(algo_plan_constrained, free_energy=true)
eval(Meta.parse(code_plan_constrained))
;

# Reference Model

In [46]:
T = 2

fg_reference = FactorGraph()

u = Vector{Variable}(undef, T)
x = Vector{Variable}(undef, T)
y = Vector{Variable}(undef, T)

@RV x_t_min ~ Categorical(placeholder(:D_t_min, dims=(8,)))

x_k_min = x_t_min
for k=1:T
    @RV u[k]
    @RV x[k] ~ Transition(x_k_min, u[k])
    @RV y[k] ~ Transition(x[k], placeholder(:A, dims=(16,8), var_id=:A_*k))

    placeholder(u[k], :u, index=k, dims=(8,8))
    Categorical(y[k], placeholder(:C, dims=(16,), index=k, var_id=:C_*k))
    # No point-mass constraint
    
    x_k_min = x[k]
end
;

In [47]:
q_plan_reference = PosteriorFactorization(fg_reference)
algo_plan_reference = messagePassingAlgorithm(y, id=:PlanReference, free_energy=true)
code_plan_reference = algorithmSourceCode(algo_plan_reference, free_energy=true)
eval(Meta.parse(code_plan_reference))
;

# Results

In [48]:
# Reward probability and utility, uncomment scenario of interest
α = 0.9; c = 2.0
# α = 0.5; c = 2.0
# α = 0.9; c = 0.0
# α = 1.0; c = 2.0

include("environment.jl")
include("agent.jl")
include("update_rules.jl")
include("helpers.jl")

(A, B, C, D) = constructABCD(α, c)
;

## Constrained results

In [49]:
CBFE = evaluatePolicies(A, B, C, D, "constrained")
plotResults(CBFE, clim=(7.0,12.0), dpi=300, highlight=minimum)
savefig("figures/F_constrained_c_$(c)_a_$(α).png")

## Reference results

In [50]:
BFE = evaluatePolicies(A, B, C, D, "reference")
plotResults(BFE, clim=(7.0,12.0), dpi=300, highlight=minimum)
savefig("figures/F_reference_c_$(c)_a_$(α).png")